In [1]:
json_path = r'./qisdax.json'

In [2]:
import json
import numpy as np
from scipy.stats import gmean
import pandas as pd

In [3]:
qisdax_data = None
with open(json_path) as fp:
  qisdax_data = json.load(fp)

In [4]:
stats = {}
for k,v in qisdax_data.items():
  avg_linear_rt = gmean(v['linear_rt'])
  avg_parallel_rt = gmean(v['parallel_rt'])
  total_cryo_parallel_rt = sum(v["cryo_parallel_rt"])
  total_aria_parallel_rt = sum(v["aria_parallel_rt"])
  slowdown = (avg_parallel_rt - avg_linear_rt) / avg_linear_rt
  depth_savings = (v['linear'] - np.sum(v['parallel'])) / v['linear']
  stats[k] = {'avg_linear_rt': avg_linear_rt, 'avg_parallel_rt': avg_parallel_rt, 'slowdown': slowdown, 'depth_savings': depth_savings, 'total_cryo_parallel_rt': total_cryo_parallel_rt, 'total_aria_parallel_rt': total_aria_parallel_rt}
print(json.dumps(stats, indent=2))

{
  "bv": {
    "avg_linear_rt": 8104187841.214809,
    "avg_parallel_rt": 13105718913.55952,
    "slowdown": 0.6171538925725328,
    "depth_savings": 0.4375,
    "total_cryo_parallel_rt": 345,
    "total_aria_parallel_rt": 2415
  },
  "dj": {
    "avg_linear_rt": 9648928288.487888,
    "avg_parallel_rt": 13917483341.895477,
    "slowdown": 0.4423864418704812,
    "depth_savings": 0.375,
    "total_cryo_parallel_rt": 505,
    "total_aria_parallel_rt": 3150
  },
  "ghz": {
    "avg_linear_rt": 3063000741.224198,
    "avg_parallel_rt": 6960689554.48575,
    "slowdown": 1.2725066503587434,
    "depth_savings": 0.2857142857142857,
    "total_cryo_parallel_rt": 340,
    "total_aria_parallel_rt": 2280
  },
  "grover": {
    "avg_linear_rt": 17584064024.174175,
    "avg_parallel_rt": 21579333854.213673,
    "slowdown": 0.22720969535523133,
    "depth_savings": 0.34210526315789475,
    "total_cryo_parallel_rt": 1315,
    "total_aria_parallel_rt": 7905
  },
  "simon": {
    "avg_linear_rt": 839

In [5]:
agg_stats = {}
agg_slowdown = tuple(v['slowdown'] for v in stats.values())
agg_stats['slowdown_mean'] = gmean(agg_slowdown)
agg_stats['slowdown_std'] = np.std(agg_slowdown, ddof=1)
agg_depth_savings = tuple(v['depth_savings'] for v in stats.values())
agg_stats['depth_savings_mean'] = gmean(agg_depth_savings)
agg_stats['depth_savings_std'] = np.std(agg_depth_savings, ddof=1)
agg_stats

{'slowdown_mean': 0.5077084684119908,
 'slowdown_std': 0.40191547382168474,
 'depth_savings_mean': 0.3596056121020118,
 'depth_savings_std': 0.055335118229876294}

In [6]:
row_names = tuple(["Bernstein-Vazirani algorithm", "Deutsch-Jozsa algorithm", "GHZ state", "Grover's algorithm", "Simon's algorithm"])
common_columns = ["Benchmark", "No restructuring", "QisDAX"]

In [7]:
df2 = pd.DataFrame(dict(benchmark=row_names,
                       no_res=map(lambda x: round(x["avg_linear_rt"] * pow(10, -6)), stats.values()),
                       qisdax=map(lambda x: round(x["avg_parallel_rt"] * pow(10, -6)), stats.values()),
                       perc=map(lambda x: x["slowdown"] * 100, stats.values())
                       )
)
df2.columns = common_columns + ["Slowdown"]
df2 = pd.concat([df2, pd.DataFrame([["Geometric Mean", None, None, agg_stats["slowdown_mean"] * 100]], columns=df2.columns)], ignore_index=True)
t2 = df2.style.format(precision=2, na_rep='').hide(axis="index").to_latex(hrules=True, label="tab:runtime", caption="Mean transpilation time [ms] with and without restructuring")
print(t2)

\begin{table}
\caption{Mean transpilation time [ms] with and without restructuring}
\label{tab:runtime}
\begin{tabular}{lllr}
\toprule
Benchmark & No restructuring & QisDAX & Slowdown \\
\midrule
Bernstein-Vazirani algorithm & 8104 & 13106 & 61.72 \\
Deutsch-Jozsa algorithm & 9649 & 13917 & 44.24 \\
GHZ state & 3063 & 6961 & 127.25 \\
Grover's algorithm & 17584 & 21579 & 22.72 \\
Simon's algorithm & 8399 & 11989 & 42.74 \\
Geometric Mean &  &  & 50.77 \\
\bottomrule
\end{tabular}
\end{table}



In [8]:
df3 = pd.DataFrame(dict(benchmark=row_names,
                       no_res=map(lambda x: x["cryo_linear_rt"], qisdax_data.values()),
                       qisdax=map(lambda x: x["total_cryo_parallel_rt"], stats.values()),
                       perc=map(lambda x: (qisdax_data[x]["cryo_linear_rt"] - stats[x]["total_cryo_parallel_rt"]) * 100 / qisdax_data[x]["cryo_linear_rt"], stats.keys())
                       )
)
df3.columns = common_columns + ["Speedup"]
df3 = pd.concat([df3, pd.DataFrame([["Geometric Mean", None, None, gmean(df3.loc[:, 'Speedup'])]], columns=df3.columns)], ignore_index=True)
t3 = df3.style.format(precision=2, na_rep='').hide(axis="index").to_latex(hrules=True, label="tab:rt1", caption="Runtime [μs] with and without restructuring")
print(t3)

\begin{table}
\caption{Runtime [μs] with and without restructuring}
\label{tab:rt1}
\begin{tabular}{lllr}
\toprule
Benchmark & No restructuring & QisDAX & Speedup \\
\midrule
Bernstein-Vazirani algorithm & 400 & 345 & 13.75 \\
Deutsch-Jozsa algorithm & 585 & 505 & 13.68 \\
GHZ state & 360 & 340 & 5.56 \\
Grover's algorithm & 1465 & 1315 & 10.24 \\
Simon's algorithm & 895 & 520 & 41.90 \\
Geometric Mean &  &  & 13.50 \\
\bottomrule
\end{tabular}
\end{table}



In [9]:
df4 = pd.DataFrame(dict(benchmark=row_names,
                       no_res=map(lambda x: x["aria_linear_rt"], qisdax_data.values()),
                       qisdax=map(lambda x: x["total_aria_parallel_rt"], stats.values()),
                       perc=map(lambda x: (qisdax_data[x]["aria_linear_rt"] - stats[x]["total_aria_parallel_rt"]) * 100 / qisdax_data[x]["aria_linear_rt"], stats.keys())
                       )
)
df4.columns = common_columns + ["Speedup"]
df4 = pd.concat([df4, pd.DataFrame([["Geometric Mean", None, None, gmean(df4.loc[:, 'Speedup'])]], columns=df4.columns)], ignore_index=True)
t4 = df4.style.format(precision=2, na_rep='').hide(axis="index").to_latex(hrules=True, label="tab:rt2", caption="Runtime [μs] with and without restructuring")
print(t4)

\begin{table}
\caption{Runtime [μs] with and without restructuring}
\label{tab:rt2}
\begin{tabular}{lllr}
\toprule
Benchmark & No restructuring & QisDAX & Speedup \\
\midrule
Bernstein-Vazirani algorithm & 3900 & 2415 & 38.08 \\
Deutsch-Jozsa algorithm & 5715 & 3150 & 44.88 \\
GHZ state & 2820 & 2280 & 19.15 \\
Grover's algorithm & 11955 & 7905 & 33.88 \\
Simon's algorithm & 6915 & 3690 & 46.64 \\
Geometric Mean &  &  & 34.89 \\
\bottomrule
\end{tabular}
\end{table}



In [10]:
micro_to_milli = 1/1000
shots = pow(10, 6)

In [11]:
df5 = pd.DataFrame(dict(benchmark=row_names,
                       no_res=map(lambda x: round(df2.iloc[x][common_columns[1]] + (micro_to_milli * shots * df3.iloc[x][common_columns[1]])), range(len(row_names))),
                       qisdax=map(lambda x: round(df2.iloc[x][common_columns[2]] + (micro_to_milli * shots * df3.iloc[x][common_columns[2]])), range(len(row_names))),
                       perc=map(lambda x: ((df2.iloc[x][common_columns[1]] + (micro_to_milli * shots * df3.iloc[x][common_columns[1]])) - (df2.iloc[x][common_columns[2]] + (micro_to_milli * shots * df3.iloc[x][common_columns[2]]))) * 100 / (df2.iloc[x][common_columns[1]] + (micro_to_milli * shots * df3.iloc[x][common_columns[1]])), range(len(row_names)))
                       )
)
df5.columns = common_columns + ["Speedup"]
df5 = pd.concat([df5, pd.DataFrame([["Geometric Mean", None, None, gmean(df5.loc[:, 'Speedup'])]], columns=df5.columns)], ignore_index=True)
t5 = df5.style.format(precision=2, na_rep='').hide(axis="index").to_latex(hrules=True, label="tab:over1", caption="Pipeline runtime [ms] with and without restructuring, $10^6$ shots, CRYO_STAQ configuration")
print(t5)

\begin{table}
\caption{Pipeline runtime [ms] with and without restructuring, $10^6$ shots, CRYO_STAQ configuration}
\label{tab:over1}
\begin{tabular}{lllr}
\toprule
Benchmark & No restructuring & QisDAX & Speedup \\
\midrule
Bernstein-Vazirani algorithm & 408104 & 358106 & 12.25 \\
Deutsch-Jozsa algorithm & 594649 & 518917 & 12.74 \\
GHZ state & 363063 & 346961 & 4.44 \\
Grover's algorithm & 1482584 & 1336579 & 9.85 \\
Simon's algorithm & 903399 & 531989 & 41.11 \\
Geometric Mean &  &  & 12.29 \\
\bottomrule
\end{tabular}
\end{table}



In [12]:
df6 = pd.DataFrame(dict(benchmark=row_names,
                       no_res=map(lambda x: round(df2.iloc[x][common_columns[1]] + (micro_to_milli * shots * df4.iloc[x][common_columns[1]])), range(len(row_names))),
                       qisdax=map(lambda x: round(df2.iloc[x][common_columns[2]] + (micro_to_milli * shots * df4.iloc[x][common_columns[2]])), range(len(row_names))),
                       perc=map(lambda x: ((df2.iloc[x][common_columns[1]] + (micro_to_milli * shots * df4.iloc[x][common_columns[1]])) - (df2.iloc[x][common_columns[2]] + (micro_to_milli * shots * df4.iloc[x][common_columns[2]]))) * 100 / (df2.iloc[x][common_columns[1]] + (micro_to_milli * shots * df4.iloc[x][common_columns[1]])), range(len(row_names)))
                       )
)
df6.columns = common_columns + ["Speedup"]
df6 = pd.concat([df6, pd.DataFrame([["Geometric Mean", None, None, gmean(df6.loc[:, 'Speedup'])]], columns=df6.columns)], ignore_index=True)
t6 = df6.style.format(precision=2, na_rep='').hide(axis="index").to_latex(hrules=True, label="tab:over2", caption="Pipeline runtime [ms] with and without restructuring, $10^6$ shots, IonQ Aria configuration")
print(t6)

\begin{table}
\caption{Pipeline runtime [ms] with and without restructuring, $10^6$ shots, IonQ Aria configuration}
\label{tab:over2}
\begin{tabular}{lllr}
\toprule
Benchmark & No restructuring & QisDAX & Speedup \\
\midrule
Bernstein-Vazirani algorithm & 3908104 & 2428106 & 37.87 \\
Deutsch-Jozsa algorithm & 5724649 & 3163917 & 44.73 \\
GHZ state & 2823063 & 2286961 & 18.99 \\
Grover's algorithm & 11972584 & 7926579 & 33.79 \\
Simon's algorithm & 6923399 & 3701989 & 46.53 \\
Geometric Mean &  &  & 34.74 \\
\bottomrule
\end{tabular}
\end{table}

